In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

# Funzioni

In [ ]:
# questa funzione ritorna True se non ci sono duplicati per quella colonna 
# altrimenti False se invece ne contiene
def check_uniqueness(df):
    for col in df.columns:
        boolean = df[col].is_unique
        print(col, boolean)

In [ ]:
def check_missing(df):

    for col in df.columns:
        print(col, (df[df[col] == '?'].shape[0] / df.shape[0]) * 100 )

# Tournament

In [ ]:
tournament_df = pd.read_csv('tournament.csv')
tournament_df.head()

In [ ]:
check_missing(tournament_df)

In [ ]:
check_duplicate(tournament_df)

imputazione dei missing values di surface con la moda

In [ ]:
tournament_df.loc[ tournament_df['surface'] == '?', 'surface' ] = tournament_df['surface'].mode()
check_missing(tournament_df)

# Match

In [ ]:
match_df = pd.read_csv('match.csv')
match_df.head()

In [ ]:
check_duplicate(match_df)

In [ ]:
check_missing(match_df)

elimino tutte le colonne con piu del 50% dei valori mancanti

In [ ]:
match_df.drop(columns= ['minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
                       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
                       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
                       'l_bpFaced'], inplace = True)

check_missing(match_df)

In [ ]:
match_df.head()

i valori winner/loser_rank_points vengono determinati usando un valore randomico normale 
parametrizzato sulla media e std_dev dei winner/loser_rank_points di ciascun torneo

In [ ]:
new_match_df = match_df[match_df['winner_rank_points'] != '?']
new_match_df = new_match_df.astype({"winner_rank_points": float})

new_match_df = new_match_df[new_match_df['loser_rank'] != '?']
new_match_df = new_match_df.astype({"loser_rank": float})

new_match_df = new_match_df[new_match_df['loser_rank_points'] != '?']
new_match_df = new_match_df.astype({"loser_rank_points": float})

new_match_df = new_match_df[new_match_df['winner_rank'] != '?']
new_match_df = new_match_df.astype({"winner_rank": float})

In [ ]:
new_match_df.info()

imputazione loser/winner rank points

In [ ]:
check_missing(match_df)

In [ ]:
from copy import copy 
def fill_rankPoints(original_df, fill_col):
    
    df = original_df.copy()
    tourneys = df['tourney_id'].unique()
    for tourney in tourneys:
        
        tourney_match_df= df[df['tourney_id']== tourney]
        
        # take only not na_values
        tourney_no_na = tourney_match_df[tourney_match_df[fill_col] != '?'].astype({fill_col: float})
        
        # getting mean and std_dev of the tourney
        mean = tourney_no_na[fill_col].mean()
        std_dev = tourney_no_na[fill_col].std()
        
        # fill ? values in df with random gaussian value
        df.loc[ ( (df[fill_col]== '?') & (df['tourney_id'] == tourney) ), fill_col] = abs(np.random.normal(mean, std_dev))
        
    return df

match_df = fill_rankPoints(match_df, 'loser_rank_points')
match_df = fill_rankPoints(match_df, 'winner_rank_points')
check_missing(match_df)

imputazione loser/winner rank basandoci sui loser/winner rank points

In [ ]:
import scipy

print('correlation between loser_rank and loser_rank_points', 
      new_match_df['winner_rank'].corr(new_match_df['winner_rank_points'], method= 'spearman' ) )

print('correlation between winner_rank and winner_rank_points', 
      new_match_df['loser_rank'].corr(new_match_df['loser_rank_points'], method= 'spearman' ) )

creo un modello di randomForestRegression per determinare i winner/loser_rank usando i winner/loser_rank_points

In [ ]:
from sklearn.model_selection import train_test_split

X = new_match_df['loser_rank_points'].values
Y = new_match_df['loser_rank'].values

x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 42)

In [ ]:
plt.title('Relationship between rank_points and rank')
plt.scatter(x_train, y_train,  color='black')
plt.xlabel('loser_rank_points')
plt.ylabel('loser_rank')
plt.show()

In [ ]:
x_train= x_train.reshape(-1, 1)
x_test = x_test.reshape(-1, 1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, r2_score

rf_loser_rank = RandomForestRegressor()
cv = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_validate(rf_loser_rank, x_train, y_train, cv= cv, scoring='r2')

print("CV-scores: %0.2f r2 with a standard deviation of %0.2f" % (scores['test_score'].mean(), scores['test_score'].std()))

In [ ]:
rf_loser_rank.fit(x_train, y_train)
y_pred = rf_loser_rank.predict(x_test)

r2_score(y_test, y_pred)

In [ ]:
def get_ranks(original_df, clf, col_in, col_out):
    df = original_df.copy()
    
    df[col_in]= df[col_in].astype({col_in: float})
    X = np.int_( np.around( df[col_in].values ) ).reshape(-1, 1)
    
    df['prediction'] = np.around(clf.predict(X))
    df.loc[ (df[col_out]== '?'), col_out ] = df.loc[ (df[col_out]== '?'), 'prediction' ]
    
    return df

match_df= get_ranks( match_df, rf_loser_rank, 'loser_rank_points', 'loser_rank' )
match_df= get_ranks( match_df, rf_loser_rank, 'winner_rank_points', 'winner_rank' )
check_missing(match_df)

imputazione della colonna score con la moda

In [ ]:
match_df.loc[match_df['score']=='?', 'score']= match_df['score'].mode()
check_missing(match_df)

In [ ]:
match_df.drop(columns= ['prediction'], inplace = True)

# Player

In [ ]:
player_df = pd.read_csv('player.csv')
player_df.head()

In [ ]:
check_missing(player_df)

In [ ]:
player_df['name'].isna().sum()

In [ ]:
check_duplicate(player_df)

poiche ci sono troppi missing values in ht, la droppiamo

In [ ]:
player_df.drop(columns= ['ht'], inplace = True)
check_missing(player_df)

ricerca di una qualche relazione tra i rank_points e il byear

In [ ]:
player_df.loc[ player_df['hand'] == '?', 'hand' ] = player_df['hand'].mode()
check_missing(player_df)

plotto un istogramma dei valori di byear

In [ ]:
byear = player_df.loc[ player_df['byear'] != '?', 'byear'].astype(int)
byear.hist(bins= 10)

In [ ]:
# aggiungo al dataframe dei giocatori il massimo punteggio ottenuto nei match
points_players = match_df.groupby(by= ['winner_id'], as_index= False)['winner_rank_points'].max()
points_players= points_players.rename(columns={"winner_id": "player_id"})
player_df= player_df.merge(points_players, on= 'player_id')

player_df.head()

In [ ]:
byear_player_df= player_df[player_df['byear'] != '?']
byear_player_df= byear_player_df.astype({"winner_rank_points": float})
byear_player_df= byear_player_df.astype({"byear": int})

In [ ]:
print('correlation between byear and max winner_rank_points', 
      byear_player_df['winner_rank_points'].corr(byear_player_df['byear'], method= 'spearman' ) )

In [ ]:
X = byear_player_df['winner_rank_points'].values
Y = byear_player_df['byear'].values

In [ ]:
plt.title('Relationship between winner_rank_points and byear')
plt.scatter(X, Y,  color='black')
plt.show()

poiche non si riesce ad individuare nessuna relazione tra l'eta e il punteggio, e poiche ci sono troppi giocatori senza byear, che e un dato che ci serve per la seconda parte del progetto, abbiamo deciso di eliminare i giocatori senza un anno di nascita

In [ ]:
player_noByear_df= player_df.loc[player_df['byear'] == '?']
player_noByear_df.head()

In [ ]:
player_df= player_df.loc[player_df['byear'] != '?']
player_df.head()

In [ ]:
check_missing(player_df)

utilizziamo player_noByear_df per eliminare tutti i match dove compaiono gli id di questi giocatori

In [ ]:
match_df.shape

In [ ]:
match_df= match_df[ ~(match_df['winner_id'].isin(player_noByear_df['player_id'])) ]
match_df= match_df[ ~(match_df['loser_id'].isin(player_noByear_df['player_id'])) ]

In [ ]:
match_df.shape

check delle inconsistenze per la chiave primaria

In [ ]:
player_df.columns

In [ ]:
player_df.drop(columns=['winner_rank_points'], inplace = True)
player_df= player_df.rename(columns={"country_id": "country"})

In [ ]:
len(player_df) == len(player_df['name'].unique())

In [ ]:
names = player_df['name']
player_df[names.isin(names[names.duplicated()])]

In [ ]:
len(player_df) -1 == len(player_df['name'].unique())

sembra non esistere inconsistenza tra il nome dei giocatori e gli ids

In [ ]:
tournament_df.columns

In [ ]:
len(tournament_df) == len(tournament_df['tourney_name'].unique())

In [ ]:
column_values = tournament_df[['tourney_id', 'tourney_name']].values.ravel()
unique_values =  pd.unique(column_values)

In [ ]:
groups = tournament_df.groupby(['tourney_id', 'tourney_name'])['tourney_date'].count()
groups

In [ ]:
len( groups[groups > 1] )

In [ ]:
len( groups[groups == 1] )

In [ ]:
len(tournament_df)

tournemanet non ha problemi di inconsistenza sulla chiave

In [ ]:
match_df.columns

In [ ]:
len(match_df) == len(match_df['match_id'].unique())

In [ ]:
match_df.to_csv('match_cleaned.csv',index=False)

In [ ]:
tournament_df.to_csv('tournament_cleaned.csv',index=False)

In [ ]:
player_df.to_csv('player_cleaned.csv',index=False)

In [ ]:
geo_df= pd.read_csv('geography_cleaned.csv')
geo_df= geo_df.rename(columns={"country_code": "country_ioc"})
geo_df.to_csv('geography_cleaned.csv', index= False)

In [ ]:
match_df= pd.read_csv('match_cleaned.csv')
player_df= pd.read_csv('player_cleaned.csv')

In [ ]:
match_df[~match_df['loser_id'].isin(player_df['player_id'])]

In [ ]:
match_df.shape

In [ ]:
match_df= match_df.loc[  match_df['loser_id'].isin(player_df['player_id']),  ]
match_df.shape

In [ ]:
match_df[~match_df['loser_id'].isin(player_df['player_id'])]

In [ ]:
match_df.head()

In [ ]:
match_df.to_csv('match_cleaned.csv',index=False)

# Geography

In [ ]:
def simple_text_preprocessor(document): 
    document = re.sub(r'[^a-zA-Z]',r' ', document)#usa solo lettere dell' alfabeto
    document = re.sub(r'\([^)]*\)', '', document)#rimuovere simboli
    document = re.sub(r"([A-Z][a-z]+)([A-Z][a-z]+)", r"\1 \2", document)#aggiungere spazi tra minuscole e maiuscole
    return document
stp = np.vectorize(simple_text_preprocessor)

Source of the open-source dataset:
https://github.com/datasets/country-codes/blob/master/data/country-codes.csv

In [ ]:
dfcountry = pd.read_csv("countries.csv")
print(dfcountry.shape)
dfcountry.info()

In [ ]:
dfcountry.shape

In [ ]:
dfcountry.head()

In [ ]:
dfcountry.isna().sum()

In [ ]:
dfcountry=dfcountry.sort_values('country_name')

In [ ]:
#Apro l open-source dataset
df_new = pd.read_csv("country_codes.csv")
print(df_new.shape)

In [ ]:
df_new.head()

In [ ]:
#Da questo nuovo df ne creo uno con le variabili che mi interessano
df_new_countries=pd.DataFrame()
df_new_countries['country_code']=df_new['IOC']
df_new_countries['country']=df_new['UNTERM English Short']
df_new_countries['language']=df_new['Languages']
df_new_countries['Region']=df_new['Region Name']

In [ ]:
df_final_geo=dfcountry.merge(df_new_countries,how='left',on='country_code')

In [ ]:
print(df_final_geo.shape)
df_final_geo.head()

In [ ]:
df_final_geo = df_final_geo.astype(str)

In [ ]:
df_final_geo['language']=stp(df_final_geo[ 'language'].values)
df_final_geo['language'] = df_final_geo['language'].str.split().str[0]#prendo solo il primo elemento, cioè la lingua

In [ ]:
df_final_geo.loc[df_final_geo['country_code']=='SGP','language']='cmn'
df_final_geo.loc[df_final_geo['country_code']=='DEU','language']='de'
df_final_geo.loc[df_final_geo['country_code']=='GRC','language']='el'
df_final_geo.loc[df_final_geo['country_code']=='UNK','language']='al'#albanese
df_final_geo.loc[df_final_geo['country_code']=='MNE','language']='mo'#montenegrin
df_final_geo.loc[df_final_geo['country_code']=='NLD','language']='nl'
df_final_geo.loc[df_final_geo['country_code']=='NGA','language']='en'
df_final_geo.loc[df_final_geo['country_code']=='PHL','language']='tl'
df_final_geo.loc[df_final_geo['country_code']=='TRI','language']='en'
df_final_geo.loc[df_final_geo['country_code']=='POC','language']='unknown'

In [ ]:
df_final_geo.drop(['country_name','Region','country'],axis=1,inplace=True)

In [ ]:
df_final_geo.head()

In [ ]:
df_final_geo.to_csv('geography_cleaned.csv',index=False)